In [1]:
import tensorflow as tf
from tensorflow import keras

2022-06-23 16:31:34.404755: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing(as_frame=True)

In [4]:
housing.frame.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [20]:
X_fulll, X_test, y_full, y_test = train_test_split(housing.data, housing.target)

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(X_fulll, y_full)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


# Define model

In [8]:
# A sequential model
simple_model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

2022-06-23 16:31:40.227030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 16:31:40.231692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 16:31:40.232288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 16:31:40.233142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [10]:
# Non-sequential
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 30)           930         ['dense_2[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_3[0][0]']            

In [ ]:
simple_model.compile(loss='mean_squared_error', optimizer='sgd')
history_1 = simple_model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
mse_test = simple_model.evaluate(X_test, y_test)
print(mse_test)

162/162 [==============================] - 0s 665us/step - loss: 0.3680
0.367960125207901


In [12]:
model.compile(loss='mean_squared_error', optimizer='sgd')
history_1 = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
182/363 [==============>...............] - ETA: 0s - loss: 1.3623

2022-06-23 16:31:55.951027: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


363/363 [==============================] - 1s 1ms/step - loss: 1.1811 - val_loss: 47.4363
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.9041 - val_loss: 17.6956
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 5.6710 - val_loss: 55.8902
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 40.9903 - val_loss: 272.7899
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 16.6844 - val_loss: 70.2082
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 17.0900 - val_loss: 15.2647
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 3.7975 - val_loss: 3.5101
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 1.5471 - val_loss: 0.6117
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4971 - val_loss: 0.5383
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5760 - val_loss: 0.5273


In [13]:
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

162/162 [==============================] - 0s 633us/step - loss: 0.4291
0.42913010716438293


In [29]:
class WideAndDeepModel(keras.Model):
    def __init__(self, unints=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(unints, activation=activation)
        self.hidden2 = keras.layers.Dense(unints, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([hidden2, input_A])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        
        return (main_output, aux_output)



In [30]:
model = WideAndDeepModel()

In [24]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

In [31]:
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [32]:
history_2 = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, 
                        validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.0869 - output_1_loss: 1.7893 - output_2_loss: 4.7648 - val_loss: 1.1495 - val_output_1_loss: 0.8490 - val_output_2_loss: 3.8535
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 1.0639 - output_1_loss: 0.8183 - output_2_loss: 3.2736 - val_loss: 0.8832 - val_output_1_loss: 0.6918 - val_output_2_loss: 2.6054
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.8699 - output_1_loss: 0.7068 - output_2_loss: 2.3381 - val_loss: 0.7690 - val_output_1_loss: 0.6406 - val_output_2_loss: 1.9251
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7852 - output_1_loss: 0.6673 - output_2_loss: 1.8471 - val_loss: 0.7078 - val_output_1_loss: 0.6104 - val_output_2_loss: 1.5848
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7368 - output_1_loss: 0.6408 - output_2_loss: 1.6012 - val_loss: 0.6710 - val_output_1_loss: 0.5886 - val_ou

In [34]:
mse = model.evaluate([X_test_A, X_test_B], y=[y_test, y_test])
mse

162/162 [==============================] - 0s 888us/step - loss: 0.5331 - output_1_loss: 0.4752 - output_2_loss: 1.0546


[0.5331146717071533, 0.4751756191253662, 1.0545669794082642]